In [11]:
#필요한 라이브러리 정의
#!pip install gradio
import gradio as gr
#!pip install gtts

from tts_final import tts
#!pip install ultralytics
import torch
from ultralytics import YOLO
import cv2
import numpy as np

In [12]:
from collections import Counter
from ultralytics import YOLO
import gradio as gr
import sys
import os
import tempfile
from PIL import Image

# tts 모듈 경로 추가
sys.path.append('/content')

from tts_final import tts  # tts 함수 임포트

# 영어로 설정된 class를 한국어로 번역 (tts를 위해서)
def wordChanger(word):
    translation_dict = {
        "can": "캔",
        "glass": "유리",
        "paper": "종이",
        "pet": "페트병",
        "plastic": "플라스틱",
        "styrofoam": "스티로폼",
        "vinyl": "비닐"
    }
    return translation_dict.get(word, word)

# YOLO 모델 로드
model = YOLO("/content/best.pt")

# 객체 감지 함수 구현
def objDetect(image):
    # 이미지를 모델에 입력으로 변환
    results = model.predict(source=image)

    # 결과에서 객체 이름 추출
    detected_objects = []
    for result in results:
        detected_objects.extend([result.names[int(cls)] for cls in result.boxes.cls])

    # 객체의 개수 세기
    object_counts = Counter(detected_objects)
    object_counts_dict = dict(object_counts)

    # 최빈값 찾기
    most_common_word, _ = object_counts.most_common(1)[0]

    # 최빈값 번역하기
    best = wordChanger(most_common_word)

    return [best]

# 이미지에서 객체를 감지하고 TTS로 변환하는 함수
def process_image(image):
    try:
        print("Saving image...")
        # 이미지를 파일로 저장
        temp_image_path = tempfile.NamedTemporaryFile(delete=False, suffix='.jpg').name
        image.save(temp_image_path)
        print(f"Image saved to {temp_image_path}")

        # 객체 감지
        detected_objects = objDetect(temp_image_path)
        print(f"Detected objects: {detected_objects}")

        # TTS 결과를 생성
        temp_audio_path = tts(detected_objects)
        print(f"TTS result saved to {temp_audio_path}")

        return temp_audio_path
    except Exception as e:
        print(f"Error in process_image: {e}")
        return str(e)

# Gradio 인터페이스 생성
with gr.Blocks(css="styles.css") as iface:
    # 타이틀 및 설명 텍스트 추가
    gr.Markdown("<div class='container'><h1>Voicena 보이시네</h1>")
    gr.Markdown("사진을 등록해주세요.</div>")

    with gr.Column():
        input_image = gr.Image(type="pil", label="Input Image")
        submit_button = gr.Button("Process Image")
        output_audio = gr.Audio(label="TTS Result", type="filepath", autoplay=True)

    submit_button.click(fn=process_image, inputs=input_image, outputs=output_audio)

    # 하단바 추가
    gr.Markdown("<div class='container'><hr>")
    gr.Markdown("© 2024 8 color bird. All rights reserved.</div>")

# 인터페이스 실행 및 공유 URL 생성
iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://25bc443500479404c5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
